In [2]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv("ecommerce_orders_dirty_10000.csv")

# --- Step B.1: Clean Order Date ---
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df = df[df['Order Date'] <= pd.Timestamp.today()]

# --- Step B.2: Convert Currency Values to USD ---
def convert_price(value):
    if pd.isnull(value):
        return 0.0
    val = str(value)
    try:
        if '€' in val:
            return round(float(re.sub(r'[^\d.]', '', val)) / 1.1, 2)
        elif '£' in val:
            return round(float(re.sub(r'[^\d.]', '', val)) * 1.25, 2)
        elif 'JPY' in val or '¥' in val:
            return round(float(re.sub(r'[^\d.]', '', val)) * 0.007, 2)
        else:
            return round(float(re.sub(r'[^\d.]', '', val)), 2)
    except:
        return 0.0

df['Unit Price'] = df['Unit Price'].apply(convert_price)
df['Shipping Fee'] = df['Shipping Fee'].apply(convert_price)
df['Currency'] = "USD"

# --- Step B.3: Clean Quantity ---
word_to_num = {
    'one': 1, 'two': 2, 'three': 3, 'four': 4,
    'five': 5, 'six': 6, 'seven': 7, 'eight': 8,
    'nine': 9, 'ten': 10
}

def clean_quantity(val):
    if pd.isnull(val):
        return 0
    val = str(val).lower()
    for word, num in word_to_num.items():
        if word in val:
            return num
    digits = re.findall(r'\d+', val)
    return int(digits[0]) if digits else 0

df['Quantity'] = df['Quantity'].apply(clean_quantity)

# --- Step B.4: Recalculate Total Price ---
df['Total Price'] = df['Quantity'] * df['Unit Price'] + df['Shipping Fee']

# --- Step B.5: Categorize Notes ---
df['Notes'] = df['Notes'].astype(str).str.lower()

def categorize_notes(note):
    if any(k in note for k in ['excellent', 'great', 'good', '👍', 'satisfied', 'happy']):
        return "Positive Feedback"
    elif any(k in note for k in ['bad', 'late', 'missing', 'angry', 'unhappy']):
        return "Negative Feedback"
    elif note.strip() == '' or note in ['nan', 'none']:
        return "No Feedback"
    else:
        return "No Feedback"

df['Notes'] = df['Notes'].apply(categorize_notes)

# --- Step B.6: Clean Product Description ---
def clean_product_description(desc):
    if pd.isnull(desc):
        return ''
    desc = re.sub(r'<[^>]+>', '', desc)  # Remove HTML tags
    desc = re.sub(r'&nbsp;', ' ', desc)  # Replace HTML non-breaking space
    desc = re.sub(r'\s+', ' ', desc)     # Normalize whitespace
    return desc.strip()

df['Product Description'] = df['Product Description'].apply(clean_product_description)

# --- Step B.7: Extract Numeric Product SKU ---
def extract_numeric_sku(sku):
    if pd.isnull(sku):
        return ''
    match = re.search(r'(\d+)', str(sku))
    return match.group(1) if match else ''

df['Product SKU Numeric'] = df['Product SKU'].apply(extract_numeric_sku)

# --- Step C.x: Generate Cleaned Email from Customer Name ---
def generate_email(name):
    if pd.isnull(name):
        return ''
    name_clean = re.sub(r'[^\w\s]', '', name.lower())  # remove special chars
    parts = name_clean.strip().split()
    if len(parts) >= 2:
        return f"{parts[0]}.{parts[1]}@email.com"
    elif len(parts) == 1:
        return f"{parts[0]}@email.com"
    else:
        return ''

df['Email'] = df['Customer Name'].apply(generate_email)

# Clean Customer Name
def clean_customer_name(name):
    if pd.isnull(name):
        return ''
    name_clean = re.sub(r'[^\x00-\x7F]+', '', name)
    name_clean = re.sub(r'[^\w\s]', '', name_clean)
    return name_clean.strip().title()

df['Customer Name'] = df['Customer Name'].apply(clean_customer_name)

# Replace Product SKU with numeric ID
# Extract numeric ID from Product SKU (e.g., 'SKU-123', '#SKU123' → 123)
# Clean and reformat Product SKU to standard "SKU-<number>"
def format_sku(sku):
    if pd.isnull(sku):
        return ''
    match = re.search(r'(\d+)', str(sku))
    return f"SKU-{match.group(1)}" if match else ''

df['Product SKU'] = df['Product SKU'].apply(format_sku)

# --- Preview ---
print(df[['Customer Name', 'Email', 'Order Date', 'Quantity', 'Unit Price', 'Shipping Fee', 'Total Price', 'Currency', 'Notes', 'Product Description', 'Product SKU Numeric']].head())

df.to_csv("ecommerce_orders_cleaned.csv", index=False)

  Customer Name                  Email Order Date  Quantity  Unit Price  \
1   Marie Curie  marie.curie@email.com 2023-05-12         1       40.00   
2          Alex         alex@email.com 2023-05-12         0     1500.00   
5    Jane Smith   jane.smith@email.com 2023-05-12         3       10.00   
6          Alex         alex@email.com 2023-05-12         1     1500.00   
7          Alex         alex@email.com 2023-05-12         1       15.62   

   Shipping Fee  Total Price Currency              Notes Product Description  \
1           7.0        47.00      USD        No Feedback          Smartphone   
2           0.0         0.00      USD  Positive Feedback          Laptop Bag   
5           0.0        30.00      USD        No Feedback      Wireless Mouse   
6           0.0      1500.00      USD        No Feedback        Gaming Chair   
7           3.0        18.62      USD  Positive Feedback        Gaming Chair   

  Product SKU Numeric  
1                 345  
2                 23

In [71]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Now print or inspect your DataFrame
print(df)


      Order ID Customer Name                  Email         Country Currency  \
1      ORD2676   Marie Curie  marie.curie@email.com             USA      USD   
2      ORD5686          Alex         alex@email.com  United Kingdom      USD   
5     ORD12739    Jane Smith   jane.smith@email.com              UK      USD   
6     ORD15843          Alex         alex@email.com              DE      USD   
7     ORD17573          Alex         alex@email.com              DE      USD   
9     ORD16610   Carlos Diaz  carlos.diaz@email.com             USA      USD   
10    ORD14612                         王伟@email.com         Germany      USD   
12     ORD8857   Carlos Diaz  carlos.diaz@email.com     Deutschland      USD   
13     ORD3050    Anna Mller  anna.müller@email.com  United Kingdom      USD   
14         NaN    Anna Mller  anna.müller@email.com   United States      USD   
15         NaN          Alex         alex@email.com              DE      USD   
16    ORD17293          Alex         ale